In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import seaborn as sns


from scipy.linalg import svd
from sklearn.metrics import mean_squared_error
from PyEMD import EMD  # pipenv install EMD-signal
from ewtpy import EWT1D  # pipenv install ewtpy
from statsmodels.tsa.stattools import acf
from torch.utils.data import DataLoader, TensorDataset
from fbm import FBM  # pip install fbm
from sklearn.decomposition import PCA

In [ ]:
X = pd.read_csv("x.csv", delimiter=",")
Y = pd.read_csv("y.csv", delimiter=",", dtype=np.float64)

print(f"Data read as: X -- {X.shape}, Y -- {Y.shape}")

In [ ]:
plt.figure(figsize=(12,12))
for i, c in enumerate(X.columns):
    plt.subplot(len(X.columns), 1, i+1)
    plt.plot(X[c])
    plt.ylabel(c)

plt.show()

In [ ]:
for c in X.columns:
    print(f"Series {c} has {sum(X[c].isna())} NaN-values")

## PCA

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
plt.figure(figsize=(8, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=Y.to_numpy(), cmap='viridis', edgecolor='k', s=50)
plt.title('PCA of X')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(label='Y')
plt.grid(True)
plt.show()

In [ ]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

def dtc_linear_regression(X, Y, test_size=.2, random_state=42, log=False):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)
    # Train a linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    # Make predictions
    y_pred = model.predict(X_test)
    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    if log: print(f"Mean Squared Error: {mse}")
    if log: print(f"R^2 Score: {r2}")
    # Plot the predictions against the true values
    if log:
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
        plt.xlabel('True Values')
        plt.ylabel('Predictions')
        plt.title('True vs Predicted Values')
        plt.grid(True)
        plt.show()

        # plt.plot(Y)
        plt.plot((Y - model.predict(X))/(Y))
    
    return mse, r2

dtc_linear_regression(X, Y, log=True)

In [ ]:
np.random.seed(42)
test_seeds = np.random.randint(1, high=4294967295, size=1000)
res = [dtc_linear_regression(X, Y, test_size=0.99, random_state=r) for r in test_seeds]

In [ ]:
mses, r2s = zip(*res)
print(f"Mean mse: {np.mean(mses):.3f}, mean r2: {np.mean(r2s):.3f}")

In [ ]:
plt.plot(np.sort(r2s))

In [ ]:
test_sizes = [.99, .95, .90, .75, .50, .25, .10, .05, .01]
plt.figure(figsize=(12,12))

for ts in test_sizes:
    np.random.seed(42)
    test_seeds = np.random.randint(1, high=4294967295, size=1000)
    res = [dtc_linear_regression(X, Y, test_size=ts, random_state=r) for r in test_seeds]
    mses, r2s = zip(*res)
    # print(f"Mean mse: {np.mean(mses):.3f}, mean r2: {np.mean(r2s):.3f}")
    plt.subplot(1, 2, 1); plt.plot(np.sort(mses), label=f"{ts}")
    plt.subplot(1, 2, 2); plt.plot(np.sort(r2s), label=f"{ts}")

plt.legend()
plt.show()

In [ ]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pca, Y, test_size=0.2, random_state=42)
# Train a linear regression model
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
# Make predictions
y_pred = model.predict(X_test)
# Evaluate the model
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")
# Plot the predictions against the true values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title('True vs Predicted Values')
plt.grid(True)
plt.show()



## EMD

In [ ]:
emd = EMD()
imfs_emd = emd(Y.DTC.to_numpy())

In [ ]:
plt.figure(figsize=(12,12))
plt.subplot(imfs_emd.shape[0]+1, 1, 1); plt.plot(Y.DTC)

for i in range(imfs_emd.shape[0]):
    plt.subplot(imfs_emd.shape[0]+1, 1, i+2)
    plt.plot(imfs_emd[i, :])

plt.show()

## EWT

In [ ]:
n_ewt_c = 5
ewt_output = EWT1D(Y.DTC, N = n_ewt_c)

In [ ]:
plt.figure(figsize=(12,12))
plt.subplot(n_ewt_c+1, 1, 1)
plt.plot(Y.DTC)

for i in range(n_ewt_c):
    plt.subplot(n_ewt_c + 1, 1, i+2)
    plt.plot(ewt_output[0][:, i])

plt.show()

## DFA

In [ ]:
def detrended_fluctuation_analysis(signal, scales, order=1):
    N = len(signal)
    Y = np.cumsum(signal - np.mean(signal))
    F = []

    for s in scales:
        n_segments = N // s
        local_rms = []

        for i in range(n_segments):
            segment = Y[i*s:(i+1)*s]
            x = np.arange(s)
            coeffs = np.polyfit(x, segment, order)
            trend = np.polyval(coeffs, x)
            rms = np.sqrt(np.mean((segment - trend)**2))
            local_rms.append(rms)

        F.append(np.sqrt(np.mean(np.square(local_rms))))

    return np.array(F)

In [ ]:
N = len(Y)
scales = np.unique(np.logspace(1.1, np.log10(N/4), num=30, dtype=int))

dfa_dtc = detrended_fluctuation_analysis(Y.DTC.to_numpy(), scales)

In [ ]:
log_scales = np.log(scales)
log_F = np.log(dfa_dtc)

alpha, intercept = np.polyfit(log_scales, log_F, 1)

plt.plot(log_scales, log_F, 'o', label="DFA")
plt.plot(log_scales, np.polyval([alpha, intercept], log_scales), linestyle="--", label=f"Fit: alpha = {alpha:.3f}")
# plt.plot(log_scales, alpha*log_scales+intercept)
plt.legend(); plt.show()

## ARIMA?

In [ ]:
dtc_diffed = np.diff(Y.DTC)
plt.plot(dtc_diffed)

In [ ]:
dfa_diffed_dtc = detrended_fluctuation_analysis(dtc_diffed, scales)
dtc_diffed_log_F = np.log(dfa_diffed_dtc)
dtc_diffed_poly = np.polyfit(log_scales, dtc_diffed_log_F, 1)

plt.plot(log_scales, dtc_diffed_log_F, 'o', label="DFA")
plt.plot(log_scales, np.polyval(dtc_diffed_poly, log_scales), label=f"Fit: alpha = {dtc_diffed_poly[0]:.3f}")
plt.legend(); plt.show()

## Power spectrum

In [ ]:
from scipy.fft import fft, dct, idct, ifft

In [ ]:
# a = fft(Y.DTC)
# a = dct(Y.DTC)
a = dct(dtc_diffed)
plt.plot(a)

## PLS regression

In [ ]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error, r2_score

def dtc_regession_monte_carlo(X, Y, test_size=0.2, random_state=42, log=False):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)
    
    model = PLSRegression(n_components=2, scale=True)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    # Evaluate the model
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    if log: print(f"Mean Squared Error: {mse}")
    if log: print(f"R^2 Score: {r2}")
    # Plot the predictions against the true values
    if log:
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
        plt.xlabel('True Values')
        plt.ylabel('Predictions')
        plt.title('True vs Predicted Values')
        plt.grid(True)
        plt.show()

        plt.plot(Y)
        plt.plot(model.predict(X))
        plt.show()

        plt.plot((Y - model.predict(X))/(Y))
        plt.show()
    
    return mse, r2

In [ ]:

np.random.seed(42)
test_seeds = np.random.randint(1, high=4294967295, size=1000)
res = [dtc_regession_monte_carlo(X, Y, test_size=.2, random_state=r) for r in test_seeds]

In [ ]:
mses, r2s = zip(*res) 
print(f"Mean mse: {np.mean(mses):.3f}, mean r2: {np.mean(r2s):.3f}")

## RPCA

In [ ]:
from rpca import RobustPCA

rpca = RobustPCA(0.1)

L, S = rpca.fit(X.to_numpy())

In [ ]:
plt.figure(figsize=(12,12))
plt.axhline(40)
X.GR.plot()

In [ ]:
plt.figure(figsize=(12,12))
plt.subplot(len(X.columns)+1, 1, 1)
plt.plot(Y)
for i, c in enumerate(X.columns):
    plt.subplot(len(X.columns)+1, 1, i+2)
    plt.plot(S[:, i])
    plt.ylabel(c)

plt.show()

In [ ]:
dtc_linear_regression(L, Y, log=True)